In [2]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# **Data Pre-Processing**

In [4]:
#mount the drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%ls


drive/  sample_data/


In [6]:
%cd /content/drive/My Drive/Group8-Project4/Resources/

/content/drive/My Drive/Group8-Project4/Resources


In [7]:
# defined path to csv
df = pd.read_csv('WELFake_Dataset.csv')

# display the first few rows of the DataFrame
df.head(10)

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
6,6,DR BEN CARSON TARGETED BY THE IRS: “I never ha...,DR. BEN CARSON TELLS THE STORY OF WHAT HAPPENE...,1
7,7,HOUSE INTEL CHAIR On Trump-Russia Fake Story: ...,,1
8,8,Sports Bar Owner Bans NFL Games…Will Show Only...,"The owner of the Ringling Bar, located south o...",1
9,9,Latest Pipeline Leak Underscores Dangers Of Da...,"FILE – In this Sept. 15, 2005 file photo, the ...",1


In [8]:
#see df info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [9]:
#identify size of table/ data set
df.shape

(72134, 4)

In [10]:
#drop the Unnamed: 0 column as we will not need it for the analysis
df.drop(['Unnamed: 0'], axis=1, inplace= True)

In [11]:
# Converting the 'title' and 'text' columns to lowercase
df['title'] = df['title'].str.lower()
df['text'] = df['text'].str.lower()

In [12]:
#check for Null values
Null_values = df.isnull().sum()
Null_values

title    558
text      39
label      0
dtype: int64

In [13]:
#drop Nan_values
df= df.fillna('')

In [14]:
#check that Nan values have been dropped correctly
df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [15]:
#print the new shape of the dataset
df.shape

(72134, 3)

In [16]:
# see the distribution of the labels real vs fake
print(f'Ratio of real and fake news:')
df['label'].value_counts(normalize=True).rename({1: 'real', 0: 'fake'})

Ratio of real and fake news:


real    0.514404
fake    0.485596
Name: label, dtype: float64

In [17]:
# combine the title and the text columns
df["title_text"] = df["title"] + df["text"]
df.head(10)

,title,text,label,title_text
0,law enforcement on high alert following threat...,no comment is expected from barack obama membe...,1,law enforcement on high alert following threat...
1,,did they post their votes for hillary already?,1,did they post their votes for hillary already?
2,unbelievable! obama’s attorney general says mo...,"now, most of the demonstrators gathered last ...",1,unbelievable! obama’s attorney general says mo...
3,"bobby jindal, raised hindu, uses story of chri...",a dozen politically active pastors came here f...,0,"bobby jindal, raised hindu, uses story of chri..."
4,satan 2: russia unvelis an image of its terrif...,"the rs-28 sarmat missile, dubbed satan 2, will...",1,satan 2: russia unvelis an image of its terrif...
5,about time! christian group sues amazon and sp...,all we can say on this one is it s about time ...,1,about time! christian group sues amazon and sp...
6,dr ben carson targeted by the irs: “i never ha...,dr. ben carson tells the story of what happene...,1,dr ben carson targeted by the irs: “i never ha...
7,house intel chair on trump-russia fake story: ...,,1,house intel chair on trump-russia fake story: ...
8,sports bar owner bans nfl games…will show only...,"the owner of the ringling bar, located south o...",1,sports bar owner bans nfl games…will show only...
9,latest pipeline leak underscores dangers of da...,"file – in this sept. 15, 2005 file photo, the ...",1,latest pipeline leak underscores dangers of da...


In [18]:
#removing all special characters AND numbers from the title_text column
import re

# create a function to remove punctuation and special characters in case I need the same function later
def remove_special_characters(text):
    return re.sub(r'[^\w\s]|[\d]', '', text)

# Apply the function to the 'title_text' column
df['title_text'] = df['title_text'].apply(remove_special_characters)

In [19]:
#see transformed df
df.head()

,title,text,label,title_text
0,law enforcement on high alert following threat...,no comment is expected from barack obama membe...,1,law enforcement on high alert following threat...
1,,did they post their votes for hillary already?,1,did they post their votes for hillary already
2,unbelievable! obama’s attorney general says mo...,"now, most of the demonstrators gathered last ...",1,unbelievable obamas attorney general says most...
3,"bobby jindal, raised hindu, uses story of chri...",a dozen politically active pastors came here f...,0,bobby jindal raised hindu uses story of christ...
4,satan 2: russia unvelis an image of its terrif...,"the rs-28 sarmat missile, dubbed satan 2, will...",1,satan russia unvelis an image of its terrifyi...


In [20]:
#remove stop words using the nltk
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the function to your column
df['title_text'] = df['title_text'].apply(remove_stopwords)


In [21]:
#check update
df.head()

,title,text,label,title_text
0,law enforcement on high alert following threat...,no comment is expected from barack obama membe...,1,law enforcement high alert following threats c...
1,,did they post their votes for hillary already?,1,post votes hillary already
2,unbelievable! obama’s attorney general says mo...,"now, most of the demonstrators gathered last ...",1,unbelievable obamas attorney general says char...
3,"bobby jindal, raised hindu, uses story of chri...",a dozen politically active pastors came here f...,0,bobby jindal raised hindu uses story christian...
4,satan 2: russia unvelis an image of its terrif...,"the rs-28 sarmat missile, dubbed satan 2, will...",1,satan russia unvelis image terrifying new supe...


In [22]:
# Creating a new DataFrame with only 'text_title' and 'labels' columns
model_df = df[['title_text', 'label']]
model_df.head()

,title_text,label
0,law enforcement high alert following threats c...,1
1,post votes hillary already,1
2,unbelievable obamas attorney general says char...,1
3,bobby jindal raised hindu uses story christian...,0
4,satan russia unvelis image terrifying new supe...,1


In [ ]:
#Exporting the model_df as a csv for easy retrival if necessary
model_df.to_csv('/content/drive/My Drive/Group8-Project4/Resources/FakeNews_Processed_Data.csv', index=False)


# **Word Embedding**

We will use Word2Vec for word embedding. We will use a pre-trained model to vectorise our data to start with because these models are trained on a massive corpus and can capture a wide range of semantic relationships. We will access the Google Word2Vec model.

In [ ]:
!pip install gensim

In [23]:
# Word2Vec expects a list of tokenized sentences, where each sentence is a list of words
#this code is tokenizing the text

from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the tokenizer model

tokenized_data = [word_tokenize(sentence) for sentence in model_df['title_text']]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [24]:
#using the pre-trained model Google word2vec

import gensim.downloader as api

w2v_model = api.load("word2vec-google-news-300")


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [25]:
# Function to create a document vector by averaging word vectors
def document_vector(word_list):
    # remove out-of-vocabulary words
    word_list = [word for word in word_list if word in w2v_model.key_to_index]
    if len(word_list) == 0:
        return np.zeros(w2v_model.vector_size)
    else:
        return np.mean(w2v_model[word_list], axis=0)

# Apply the function to each document in your tokenized data
X = np.array([document_vector(document) for document in tokenized_data])



In [26]:
!pip install h5py


In [28]:
import h5py

# Define the file path for the HDF5 file
hdf5_path = '/content/drive/My Drive/Group8-Project4/document_vectors.h5'

# Save the array to an HDF5 file
with h5py.File(hdf5_path, 'w') as hdf:
    # Create a dataset in the file
    hdf.create_dataset("document_vectors", data=X)

###IF I WANT TO LOAD THE DATA, I NEED TO USE THE BELOW CODE:
# import h5py

# # Open the HDF5 file and read data
# with h5py.File(hdf5_path, 'r') as hdf:
#     # Load the data
#     loaded_X = hdf['document_vectors'][:]

